In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,27583
2,Huelva,Confirmados PDIA 14 días,1343
3,Huelva,Tasa PDIA 14 días,"261,70664692012394"
4,Huelva,Confirmados PDIA 7 días,630
5,Huelva,Total Confirmados,27757
6,Huelva,Curados,23064
7,Huelva,Fallecidos,360


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  27583.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  7180.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 888 personas en los últimos 7 días 

Un positivo PCR cada 475 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,27583.0,630.0,1343.0,513170.0,122.766335,261.706647,130.0
Condado-Campiña,8468.0,312.0,685.0,156231.0,199.704284,438.453316,62.0
Huelva-Costa,16251.0,263.0,545.0,289548.0,90.831227,188.224405,61.0
Huelva (capital),7180.0,162.0,303.0,143837.0,112.627488,210.655117,46.0
Almonte,1365.0,117.0,240.0,24507.0,477.414616,979.312033,26.0
Moguer,1026.0,42.0,87.0,21867.0,192.070243,397.859789,6.0
Bollullos Par del Condado,746.0,22.0,43.0,14387.0,152.915827,298.880934,6.0
Sierra de Huelva-Andévalo Central,2448.0,47.0,90.0,67391.0,69.742250,133.548990,6.0
Palma del Condado (La),1022.0,11.0,21.0,10801.0,101.842422,194.426442,6.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Rociana del Condado,475.0,33.0,83.0,7939.0,415.669480,1045.471722,4.0
Almonte,1365.0,117.0,240.0,24507.0,477.414616,979.312033,26.0
Cartaya,1221.0,64.0,147.0,20083.0,318.677488,731.962356,5.0
San Bartolomé de la Torre,282.0,8.0,27.0,3761.0,212.709386,717.894177,2.0
Lucena del Puerto,191.0,7.0,22.0,3261.0,214.658080,674.639681,3.0
Almonaster la Real,62.0,5.0,9.0,1788.0,279.642058,503.355705,0.0
Cumbres Mayores,33.0,8.0,8.0,1749.0,457.404231,457.404231,5.0
Condado-Campiña,8468.0,312.0,685.0,156231.0,199.704284,438.453316,62.0
Palos de la Frontera,560.0,15.0,51.0,11742.0,127.746551,434.338273,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Alosno,161.0,1.0,6.0,3933.0,25.425884,152.555301,0.0,0.166667
Aljaraque,981.0,9.0,32.0,21474.0,41.911148,149.017416,4.0,0.281250
Palos de la Frontera,560.0,15.0,51.0,11742.0,127.746551,434.338273,0.0,0.294118
San Bartolomé de la Torre,282.0,8.0,27.0,3761.0,212.709386,717.894177,2.0,0.296296
Punta Umbría,687.0,4.0,13.0,15355.0,26.050147,84.662976,1.0,0.307692
Aracena,499.0,9.0,29.0,8255.0,109.024833,351.302241,0.0,0.310345
Lucena del Puerto,191.0,7.0,22.0,3261.0,214.658080,674.639681,3.0,0.318182
Aroche,87.0,1.0,3.0,3054.0,32.743942,98.231827,0.0,0.333333
Villalba del Alcor,471.0,1.0,3.0,3366.0,29.708853,89.126560,0.0,0.333333
